In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
# import MySQLdb as mdb
import numpy as np
import param
import panel as pn
import os
from io import StringIO

# config
pn.extension()

In [2]:
def callback_load_dataframe(target, event):
    file = event.new
    result = str(file, 'utf-8')
    data = StringIO(result)
    target.value = pd.read_csv(data)

widget_df_load = pn.widgets.DataFrame()
widget_df_load.height=500
widget_df_load_copy = pn.widgets.DataFrame()
widget_file_input = pn.widgets.FileInput()
widget_file_input.width = 400    
widget_file_input.link(widget_df_load, callbacks={'value': callback_load_dataframe})
widget_file_input.link(widget_df_load_copy, callbacks={'value': callback_load_dataframe})


Watcher(inst=FileInput(width=400), cls=<class 'panel.widgets.input.FileInput'>, fn=<function Reactive.link.<locals>.link at 0x000001CCF90BE678>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

In [60]:
# cross selector
def callback_cross_select_ticker(target, event):
    target.value = widget_df_load_copy.value.loc[widget_df_load_copy.value['ticker'].isin(event.new)]

widget_cross_selector_ticker = pn.widgets.CrossSelector(name='Ticker', value=[], 
                                                        options=list(widget_df_load_copy.value.ticker.unique()))
widget_cross_selector_ticker.link(widget_df_load, callbacks={'value': callback_cross_select_ticker})


Watcher(inst=CrossSelector(name='Ticker', options=['PNR', 'ALLE', ...]), cls=<class 'panel.widgets.select.CrossSelector'>, fn=<function Reactive.link.<locals>.link at 0x000001CCFD568D38>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

In [61]:
def callback_price_to_value_filter(target, event):
    if event.new == 'All':
        target.value = widget_df_load_copy.value
    if event.new == '[0, 1]':
        target.value = widget_df_load_copy.value.loc[(widget_df_load_copy.value['price_to_value'] > 0 ) & (widget_df_load_copy.value['price_to_value'] <= 1)]
    if event.new == '[1, 2]':
        target.value = widget_df_load_copy.value.loc[(widget_df_load_copy.value['price_to_value'] > 1 ) & (widget_df_load_copy.value['price_to_value'] <= 2)]
    if event.new == '[2, 3]':
        target.value = widget_df_load_copy.value.loc[(widget_df_load_copy.value['price_to_value'] > 2 ) & (widget_df_load_copy.value['price_to_value'] <= 3)]
    if event.new == '[3, inf]':
        target.value = widget_df_load_copy.value.loc[(widget_df_load_copy.value['price_to_value'] > 3 ) ]
    if event.new == '[-inf, 0]':
        target.value = widget_df_load_copy.value.loc[(widget_df_load_copy.value['price_to_value'] <= 0)]
    if event.new == 'NaN':
        target.value = widget_df_load_copy.value.loc[np.isnan(widget_df_load_copy.value['price_to_value'])]

widget_select_price_to_value = pn.widgets.Select(name='Price to Value', options=['All', '[0, 1]', '[1, 2]', '[2, 3]', '[3, inf]', '[-inf, 0]', 'NaN'])
widget_select_price_to_value.link(widget_df_load, callbacks={'value': callback_price_to_value_filter})

Watcher(inst=Select(name='Price to Value', options=['All', '[0, 1]', ...], value='All'), cls=<class 'panel.widgets.select.Select'>, fn=<function Reactive.link.<locals>.link at 0x000001CCFD0FEE58>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

In [64]:
# reset button
def callback_reset_dataframe(event):
    widget_cross_selector_ticker.value = list()
    widget_df_load.value = widget_df_load_copy.value
    


widget_button_reset = pn.widgets.Button(name='Reset', button_type='primary')
widget_button_reset.on_click(callback_reset_dataframe)

In [66]:
panel = pn.Row(pn.Column(widget_file_input, widget_cross_selector_ticker, widget_select_price_to_value, widget_button_reset), widget_df_load)
#panel.show()
panel

Row
    [0] Column
        [0] FileInput(filename='20210509_valuation_result..., mime_type='application/vnd.ms-excel'..., value=b'ticker,current_price,e..., width=400)
        [1] CrossSelector(name='Ticker', options=['PNR', 'ALLE', ...])
        [2] Select(name='Price to Value', options=['All', '[0, 1]', ...], value='[3, inf]')
        [3] Button(button_type='primary', clicks=1, name='Reset')
    [1] DataFrame(height=500, selection=[3], value=    ticker  current_price ...)